<a href="https://colab.research.google.com/github/AzulBarr/Aprendizaje-Automatico/blob/main/TPs/tp2/tp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import datasets
import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import classification_report, ConfusionMatrixDisplay
#from datasets import load_dataset
#from scipy.spatial.distance import cosine

In [ ]:
model_name = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
model.eval()

In [ ]:
def obtener_embedding_de_parrafo(texto: str):
  """
  Devuelve el embedding (estático) para el token.
  """
  tokens = tokenizer.tokenize(texto)
  embeddings = []
  for token in tokens:
    token_id = tokenizer.convert_tokens_to_ids(token)
    if token_id is None or token_id == tokenizer.unk_token_id:
      print(f"❌ El token '{token}' no pertenece al vocabulario de multilingual BERT.")
      return None
    embedding_vector = model.embeddings.word_embeddings.weight[token_id]
    embeddings.append(embedding_vector)
    print(f"✅ Token: '{token}' | ID: {token_id}")
    print(f"Embedding shape: {embedding_vector.shape}")
  return embeddings

## Cargamos los data sets y los armamos con las etiquetas que necesitamos.

tener en cuenta que sacó [], {}, @, «, etc

In [ ]:
dataset = datasets.load_dataset(path="/content/data")

In [ ]:
X_train = dataset["train"]["entrada"]
y_train = dataset["train"]["salida"]
X_test = dataset["test"]["entrada"]
y_test = dataset["test"]["salida"]

X_train = np.array([item for item in X_train])
y_train = np.array([item for item in y_train])
X_test = np.array([item for item in X_test])
y_test = np.array([item for item in y_test])

In [ ]:
def transformar_etiqueta(y):
  puntuacion_iniciales = []
  puntuacion_finales = []
  capitalizaciones = []
  instancia_id_count = 0
  instancia_ids = []
  token_ids = []
  tokens_l = []

  for parrafo in y:
    inicio_pregunta = False
    palabras = parrafo.split()


    for palabra in palabras:
      tokens = tokenizer.tokenize(palabra.lower())

      for i in range(len(tokens)):
        if tokens[i] == "¿":
          inicio_pregunta = True
          continue
        if tokens[i] == "?" or tokens[i] == "." or tokens[i] == ",":
          continue

        instancia_ids.append(instancia_id_count)
        token_ids.append(tokenizer.convert_tokens_to_ids(tokens[i]))
        tokens_l.append(tokens[i])

        if inicio_pregunta:
          puntuacion_iniciales.append('"¿"')
          inicio_pregunta = False
        else:
          puntuacion_iniciales.append("")

        if i != len(tokens) - 1:

          if tokens[i+1] == "?":
            puntuacion_finales.append('"?"')
          elif tokens[i+1] == ".":
            puntuacion_finales.append('"."')
          elif tokens[i+1] == ",":
            puntuacion_finales.append('","')
          else:
            puntuacion_finales.append("")

        else:
          puntuacion_finales.append("")

        if palabra.islower():
          capitalizaciones.append(0)
          ultimo_numero = 0
        elif palabra.istitle():
          capitalizaciones.append(1)
          ultimo_numero = 1
        elif palabra.isupper():
          capitalizaciones.append(3)
          ultimo_numero = 3
        else:
          capitalizaciones.append(2)
          ultimo_numero = 2

    instancia_id_count += 1
  etiquetas = np.column_stack([instancia_ids, token_ids, tokens_l, puntuacion_iniciales, puntuacion_finales, capitalizaciones])
  return etiquetas

In [ ]:
etiquetas_train = transformar_etiqueta(y_train)
etiquetas_test = transformar_etiqueta(y_test)

###A la función trasformar_etiquetas solo se le pueden pasar un vector de strings, no el string directamente.